In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from common import *
import os
from sklearn.metrics import classification_report
import tarfile
import glob
from sentence_transformers import SentenceTransformer

In [38]:
"""if not os.path.exists('data.csv'):
    if not os.path.exists('Fake.csv') and not os.path.exists('True.csv'):
        download_data()
    label_data('Fake.csv', 'fake_labeled.csv', 1)
    label_data('True.csv', 'true_labeled.csv', 0)
    os.system("awk '(NR == 1) || (FNR > 1)' *labeled.csv > data.csv")"""

'if not os.path.exists(\'data.csv\'):\n    if not os.path.exists(\'Fake.csv\') and not os.path.exists(\'True.csv\'):\n        download_data()\n    label_data(\'Fake.csv\', \'fake_labeled.csv\', 1)\n    label_data(\'True.csv\', \'true_labeled.csv\', 0)\n    os.system("awk \'(NR == 1) || (FNR > 1)\' *labeled.csv > data.csv")'

In [39]:
"""df = pd.read_csv('data.csv', usecols=['text', 'label'], encoding='utf-8')
df = df.sample(n=NUM_SAMPLES, random_state=RANDOM_SEED)
df.head()"""

"df = pd.read_csv('data.csv', usecols=['text', 'label'], encoding='utf-8')\ndf = df.sample(n=NUM_SAMPLES, random_state=RANDOM_SEED)\ndf.head()"

In [40]:
if not os.path.exists('aclImdb'):
    if not os.path.exists('aclImdb_v1.tar.gz'):
        files = download_data()
    else:
        files = ['aclImdb_v1.tar.gz']
    
    tarfile.open(files[0]).extractall()

paths = glob.glob('aclImdb/**/pos/*.txt', recursive=True)
pos_frame = get_data(paths, 1)
paths = glob.glob('aclImdb/**/neg/*.txt', recursive=True)
neg_frame = get_data(paths, 0)

df = pd.concat([pos_frame, neg_frame])
print(f'Size of dataset: {df.size}')
df.head()

Size of dataset: 100000


,text,label
0,Tromeo and Juliet is perhaps the best Shakespe...,1
1,"The book ""The Railway Children"" is a children'...",1
2,"If you're into alternate realities, contemplat...",1
3,"If not the best movie ever made, ""Babette's Fe...",1
4,SPOILERS BELOW<br /><br />`A Dog's Life' was m...,1


In [41]:
df = df.sample(n=NUM_SAMPLES, random_state=RANDOM_SEED)
df.head()

,text,label
1247,"As a kid, this movie scared me green. As an ad...",0
10067,It is a movie which sheds the light on the beg...,0
9590,Talk about being boring!<br /><br />I got this...,0
16668,One of Disney's best films that I can enjoy wa...,1
12196,"Being a bit of a connoisseur of garbage, I hav...",1


In [42]:
def train_models(X_tr, y_tr, X_ts, y_ts, models):
    for model in models:
       
        # print("Probability: ")
        # proba = model.predict_proba(X_ts)
        # print(proba)

        score = cross_val_score(model, X_tr, y_tr, cv=5)
        msg = ("{0}:\n\tMean accuracy on development set\t= {1:.3f} "
            "(+/- {2:.3f})".format(model.__class__.__name__,
                                    score.mean(),
                                    score.std()))
        print(msg)
        
        model.fit(X_tr, y_tr)
        pred_eval = model.predict(X_ts)
        acc_eval = accuracy_score(y_ts, pred_eval)
        print("\tAccuracy on evaluation set\t\t= {0:.3f}".format(acc_eval))
        # print(classification_report(y_ts, pred_eval))

In [43]:
ensamble_models = [ RandomForestClassifier(random_state=RANDOM_SEED),
                    GradientBoostingClassifier(random_state=RANDOM_SEED),
                    AdaBoostClassifier(random_state=RANDOM_SEED)    ]

models = [  LogisticRegression(), 
            DecisionTreeClassifier()  ]

In [44]:
countVectorizer = CountVectorizer()
embeddings_countVectorizer = preprocess_data(df['text'], countVectorizer)

X_tr, X_ts, y_tr, y_ts= train_test_split(embeddings_countVectorizer, df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

print(f'Train shape: {X_tr.shape}')
print(f'Test shape: {X_ts.shape}')

Train shape: (2500, 51072)
Test shape: (2500, 51072)


In [45]:
train_models(X_tr, y_tr, X_ts, y_ts, ensamble_models)

RandomForestClassifier:
	Mean accuracy on development set	= 0.812 (+/- 0.012)
	Accuracy on evaluation set		= 0.817
GradientBoostingClassifier:
	Mean accuracy on development set	= 0.793 (+/- 0.012)
	Accuracy on evaluation set		= 0.805
AdaBoostClassifier:
	Mean accuracy on development set	= 0.769 (+/- 0.016)
	Accuracy on evaluation set		= 0.778


In [46]:
train_models(X_tr, y_tr, X_ts, y_ts, models)

/home/justus/MA-540-TEAM3/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/justus/MA-540-TEAM3/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

LogisticRegression:
	Mean accuracy on development set	= 0.832 (+/- 0.006)


/home/justus/MA-540-TEAM3/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


	Accuracy on evaluation set		= 0.839
DecisionTreeClassifier:
	Mean accuracy on development set	= 0.701 (+/- 0.012)
	Accuracy on evaluation set		= 0.694


In [47]:
tfidfVectorizer = TfidfVectorizer()
embeddings_Tfidf = preprocess_data(df['text'], tfidfVectorizer)

X_tr, X_ts, y_tr, y_ts= train_test_split(embeddings_Tfidf, df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

print(f'Train shape: {X_tr.shape}')
print(f'Test shape: {X_ts.shape}')

Train shape: (2500, 51072)
Test shape: (2500, 51072)


In [48]:
train_models(X_tr, y_tr, X_ts, y_ts, ensamble_models)

RandomForestClassifier:
	Mean accuracy on development set	= 0.788 (+/- 0.013)
	Accuracy on evaluation set		= 0.799
GradientBoostingClassifier:
	Mean accuracy on development set	= 0.782 (+/- 0.007)
	Accuracy on evaluation set		= 0.805
AdaBoostClassifier:
	Mean accuracy on development set	= 0.767 (+/- 0.013)
	Accuracy on evaluation set		= 0.778


In [49]:
train_models(X_tr, y_tr, X_ts, y_ts, models)

LogisticRegression:
	Mean accuracy on development set	= 0.841 (+/- 0.013)
	Accuracy on evaluation set		= 0.850
DecisionTreeClassifier:
	Mean accuracy on development set	= 0.667 (+/- 0.015)
	Accuracy on evaluation set		= 0.685


In [50]:
sbert_vectorizer = SentenceTransformer('all-MiniLM-L6-v2')

df['text'] = df['text'].str.lower()
df['text'] = df['text'].apply(
    lambda text: text.translate(
        str.maketrans('', '', string.punctuation)
    )
)

X_tr, X_ts, y_tr, y_ts = train_test_split(df['text'], df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)
X_tr = sbert_vectorizer.encode(X_tr.values.astype('U'), normalize_embeddings=True)
X_ts = sbert_vectorizer.encode(X_ts.values.astype('U'), normalize_embeddings=True)

train_models(X_tr, y_tr, X_ts, y_ts, ensamble_models)


RandomForestClassifier:
	Mean accuracy on development set	= 0.767 (+/- 0.013)
	Accuracy on evaluation set		= 0.773
GradientBoostingClassifier:
	Mean accuracy on development set	= 0.768 (+/- 0.023)
	Accuracy on evaluation set		= 0.767
AdaBoostClassifier:
	Mean accuracy on development set	= 0.724 (+/- 0.011)
	Accuracy on evaluation set		= 0.736


In [51]:
train_models(X_tr, y_tr, X_ts, y_ts, models)

LogisticRegression:
	Mean accuracy on development set	= 0.808 (+/- 0.017)
	Accuracy on evaluation set		= 0.808
DecisionTreeClassifier:
	Mean accuracy on development set	= 0.610 (+/- 0.019)
	Accuracy on evaluation set		= 0.628
